# CSD 5: Tree Models 

## instructions

our course will be using an automatic grading system. <br>
after each question there will appear a code block with some prepared code to add your answer to a dictionary that will be sent to the course server for grading. <br>
please do not edit any code other than in placeholders marked `#### your code here ####` <br>
__don't forget to run the code block after you write your answer.__


you can add code blocks wherever you want in order to interact with datasets and play with your own code. <br>
in the next code block plase fill in your id number and email account in the appropriate placees. <br>
and __don't forget to run the block!__

In [ ]:
ans = {}
ans['HW'] = 'CSD5'
ans['id_number'] = #### your id here ####

0. For this Case Study assignment you should have in your current folder the ebay_boys_girls_shirts folder, holding the four CSV files describing the train and test shirts images, and the boys and girls images folders. This is what we did in CSD 1, **if you already have the data in your current folder you don't need to run this again!**:

In [ ]:
import requests
import tarfile

url = "http://www.tau.ac.il/~saharon/DScourse/ebay_boys_girls_shirts.tar.gz"
r = requests.get(url)

with open("ebay_boys_girls_shirts.tar", "wb") as file:
    file.write(r.content)

with tarfile.open("ebay_boys_girls_shirts.tar") as tar:
    tar.extractall('.')

1. In CSD4 we got the `x_train` and `x_test` matrices, and the `y_train` and `y_test` 0/1 vectors (0 = boys, 1 = girls).

    Complete the `get_final_matrices` function using all functions we've composed, to quickly get the four elements:

In [ ]:
import sys
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage import transform, color, img_as_ubyte

%matplotlib inline

def get_file_list(df, folder, n_sample = None, seed = None):
    if n_sample is None:
        file_ids_list = df.file_id.values
    else:
        file_ids_list = df.sample(n = n_sample, random_state = seed).file_id.values
    files_list = [folder + '/' + str(file_id) + '.jpg' for file_id in file_ids_list]
    return files_list

def read_image_and_resize(f, w = 100, h = 100):
    img = plt.imread(f)
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        img = transform.resize(img, (w, h), mode='constant')
        img = img_as_ubyte(img)
    img = color.gray2rgb(img)
    img = img[np.newaxis, :, :, :3]
    if img.shape != (1, 100, 100, 3):
        raise ValueError(f + str(img.shape))
    return img

def read_images_4d_array(files_list):
    images_list = [read_image_and_resize(file) for file in files_list]
    images_array = np.concatenate(images_list)
    return images_array

def get_images_matrix(csv_file, folder, n = None, seed = 1976):
    df = pd.read_csv(csv_file)
    files_list = get_file_list(df, folder, n, seed)
    images = read_images_4d_array(files_list)
    return images, files_list

def get_all_pixels(x):
    return x.reshape(-1, np.prod(x.shape[1:]))

def numpy_array_size_in_bytes(a):
    return a.size * a.itemsize

def shape_and_size(x, name):
    n_rows = x.shape[0]
    if len(x.shape) == 1:
        n_cols = 1
    elif len(x.shape) == 2:
        n_cols = x.shape[1]
    else:
        warnings.warn('Function is meaningful for 1 or 2-D numpy arrays, taking 2nd dimension as n_cols')
        n_cols = x.shape[1]        
    size = numpy_array_size_in_bytes(x)
    print('%s Shape: %d X %d, Size (bytes): %d' % (name, n_rows, n_cols, size))

def conf_matrix(y_true, y_pred):
    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

def get_final_matrices(n_train = None, n_test = None):
    folder = 'ebay_boys_girls_shirts/'
    x_boys_train, boys_train_files = get_images_matrix(folder + 'boys_train.csv', folder + 'boys', n_train)
    ### YOUR CODE HERE ###
    
    return x_train, x_test, y_train, y_test

Upload 20% of the data:

In [ ]:
x_train, x_test, y_train, y_test = get_final_matrices(n_train = 2000, n_test = 500)

Make sure you got what you wanted:

In [ ]:
shape_and_size(x_train, 'x_train')
shape_and_size(x_test, 'x_test')
shape_and_size(y_train, 'y_train')
shape_and_size(y_test, 'y_test')

1. start by training a single classification tree model. call it "mod_0", and report the model's accuracy score (on the test set. 

In [ ]:
from sklearn.tree import ### your code here ###

mod_0 = ### your code here ###(random_state=0)
### your code here ###

acc = ### your code here ###
print('Test accuracy on 3 channels with CART: %.3f' % acc)

did the single tree classifier generalize well to unseen data?<br>
take a deeper look using the scikit learn classification report.<br>
make sure you understand why a single tree is a problematic model.

train a few more single trees (all with random_state=0 !)<br>
this time limit each tree's growth,<br>
use these hyper parameter setting:<br>
__tree_1)__ max_depth = 2<br>
__tree_2)__ max_depth = 4  <br>
__tree_3)__ min_samples_split = 50

are these models any better? make sure you understand why.

for the best performing model, report the following:
#### Q1) accuracy
#### Q2) precision
#### Q3) recall

In [ ]:
ans['Q1'] = ### your answer here ###
ans['Q2'] = ### your answer here ###
ans['Q3'] = ### your answer here ###

2. Now train a Random Forest model with 100 trees and max tree depth of 4, see the confusion matrix and accuracy for a default score cutoff of 0.5: 

do not forget to use a random seed (random_state) of 0, this ensures reproducability of the results. 

In [ ]:
from sklearn.ensemble import ### your code here ###

mod = ### your code here ###(random_state=0)
### your code here ###
acc = ### your code here ###
ans['Q4'] = acc

print('Test accuracy on all pixels with RF: %.3f' % acc)

In [ ]:
y_pred = mod.predict(x_test)
y_test_s = np.array(['boys'] * len(y_test))
y_pred_s = np.array(['boys'] * len(y_test))
y_test_s[y_test == 1] = 'girls'
y_pred_s[y_pred == 1] = 'girls'
conf_matrix(y_test_s, y_pred_s)

3. See the distribution of the images score which may be interpreted as P(label(image) = boys):

In [ ]:
y_pred_prob = mod.predict_proba(x_test)

plt.hist(y_pred_prob[:, 0])
plt.xlabel('P(label(image) = boys)')
plt.ylabel('Frequency')
plt.show()

3. See if accuracy improves on a subset of test images the model is more "sure of", i.e. it scores them very high or very low:

In [ ]:
from sklearn.metrics import confusion_matrix

y_pred_prob_1d = y_pred_prob[:, 0]
y_pred_low_high_id = np.argwhere(( y_pred_prob_1d <= 0.1) | (y_pred_prob_1d >= 0.9)).reshape(-1)
y_pred_low_high = y_pred_prob[y_pred_low_high_id, 0]
y_pred_low_high_temp = y_pred_low_high.copy()
y_pred_low_high[y_pred_low_high_temp < 0.5] = 1
y_pred_low_high[y_pred_low_high_temp > 0.5] = 0
y_test_low_high = y_test[y_pred_low_high_id]

conf = confusion_matrix(y_test_low_high, y_pred_low_high)
acc = (conf[0, 0] + conf[1, 1]) / len(y_test_low_high)
print('Test accuracy on all pixels with RF, only %.2f of images with very low/high scores: %.3f' % (len(y_test_low_high)/len(y_test), acc))

5. Even with a balanced training set with two classes each having a share of 50% of the dataset, a cutoff score of 0.5 may not be optimal. Can you write a function which would receive the score `y_pred_prob_1d`, the score for classifying "boys", `y_test` the 0/1 labels, and return the cutoff score which reaches maximum accuracy and that accuracy?

In [ ]:
def optimal_cutoff_accuracy(y_test, y_pred_prob_1d):
    optimal_cutoff = 0.0
    max_accuracy = 0.0
    ### YOUR CODE HERE ###
    return optimal_cutoff, max_accuracy

optimal_cutoff, max_accuracy = optimal_cutoff_accuracy(y_test, y_pred_prob_1d)

ans['Q5'] = optimal_cutoff
ans['Q6'] = max_accuracy
print('optimal cutoff: %.2f' % optimal_cutoff)
print('max test accuracy: %.2f' % max_accuracy)

**IMPORTNAT**: the correct way of choosing a cutoff score, which is another model parameter, is choosing it on a different *validation* dataset, not your final *testing* dataset. Otherwise you stand the risk of overfitting to your final testing set.

limiting max tree depth saves a lot of time when training and predicting with RF. <br>
what will happen if we do not limit tree depth? which model would perform better? why?<br>
let's try this out. build a new RF model without tree depth limitations (on 100 trees, and random_state=0)

In [ ]:
from sklearn.ensemble import ### your code here ###

mod_2 = ### your code here ###(random_state=0)
### your code here ###
acc = ### your code here ###
ans['Q7'] = acc

print('Test accuracy on all pixels with RF: %.3f' % acc) 

as before, create a 1d array (similar to y_pred_prob_1d) using the new RF model. <br>
give it a different name, you will need to use both arrays soon.

In [ ]:
### your code here ###

6. Once you accept the possibility of different thresholds, comparing between models based on a single default cutoff score such as 0.5, is wrong.

    It is custom to draw a [ROC curve](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) of the model's performance on a **validation** dataset and use the Area Under Curve ([AUC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic#Area_under_the_curve)) to choose the best model to use on the final testing dataset. The AUC of a two-way classification model ranges between 0.5 (completely random guess) and 1 (completely correct), so the higher the AUC the better the model.

    The most common version of the ROC curve draws a line formed by comparing the following metrics with different cutoff scores:
    * True Positive Rate (TPR) which is what we have defined as Recall in CSD4: P(predict boys | label boys), you want this high
    * False Postive Rate (FPR), which is: P(predict boys | label girls), you want this low

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# Compute fpr, tpr, thresholds and roc auc
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob_1d, pos_label=0)
roc_auc = roc_auc_score(1 - y_test, y_pred_prob_1d)
ans['Q8'] = roc_auc

# Plot ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

The above is the ROC curve for the Random Forests model, the AUC is printed in the plot.

Can you add the second RF model ROC curve up there with a different color and compare the two AUCs?

In [ ]:
fpr_rf_1, tpr_rf_1, thresholds = roc_curve(y_test, y_pred_prob_1d, 0)
roc_auc_rf_1 = roc_auc_score(1 - y_test, y_pred_prob_1d)

### YOUR CODE HERE ###
ans['Q8'] = ### your code here (AUC for second RF model) ###

plt.plot(fpr_rf_1, tpr_rf_1, label='ROC curve RF (area = %0.3f)' % roc_auc_rf, color = 'blue')
### YOUR CODE HERE ###
plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

given the comparison between single tree models you have made at the begining of this exercise, <br>
do the results for RF hyper parameter tuning seem trivial? <br>
make sure you understand what is happening here.

# WOW!!!!
you have trained and compared two advanced models for classification! you're awesome!

now let's enjoy a few more theoretical questions to make sure you got things figured out

# theory

#### Q9)
When building a tree, each split is based on a (j,s)(j,s) pair of a splitting variable and a threshold (splitting condition). Does it make sense for a tree building algorithm to perform several different splits on the same variable in the same tree?

1. No, it's impossible and meaningless
2. No, because the relevant information in the variable is used in the first split on it
3. Yes, as long as multiple splits on the same variable are not on the same path in the tree (meaning they will never both be checked for the same observation)
4. Yes, after splitting once on a variable, a second split on the same variable can still be informative



In [ ]:
ans['Q9'] = #### your answer here ####

#### Q10)
Assume we have a fixed random forest (RF) algorithm, where parameters like tree size, randomization methods of samples and splitting decisions, etc. are fixed in advance. We can only play with the number of trees built and averaged. 
How does accuracy of RF predictions (on test set or new prediction task) depend on the number of trees built? 

1. The dependence has the familiar "U-shape", where selecting a good number of trees will give more accurate prediction than too few or too many
2. Predictive performance in general improves with more trees, and the only reason to limit the number of trees in RF is computational
3. The first answer applies when trees are small, the second when they are big
4. RF is never a more accurate predictor than a single tree, so one is the best RF size for prediction

In [ ]:
ans['Q10'] = #### your answer here ####

#### Q11)
Assume we have a problem with a training set and test set obtained by randomly dividing data as we did in class. 
We said in class that the 1-NN has perfect performance on the training set (for both regression and classification), because an observation is its own nearest neighbor. Imagine now that we implement 1-NN on the training set in such a way that for each observation we find the closest neighbor that is *different* than the observation itself (that is, for observation $i$ in the training set we find $j_0$ such that $x_{j_0}$ is the closest neighbor of $x_i$ among all $x_j,\;j\neq i.$
What can we now say about the training set predictive perforamcne of this modified 1-NN method, compared to its test set performance?

1. They will be similar, because what we do for training set observations is now similar to what we do on the test set
2. Training set performance will still be much better, since the model is learned on the training set only
3. The performance on the training will be similar to that of 2-NN on the test set
4. Because this is not standard 1-NN, we cannot say anything about the performance

Enrichment note: What we have described here is in fact doing "leave-one-out cross validation" on the training set, our first encounter with this very important method. 


In [ ]:
ans['Q11'] = #### your answer here ####

#### Q12)
We mentioned bagging and RF use bootstrap sampling of $n$ out of $n$ observations in training set with return. This means that some observations get  selected 0 times, some 1, some 2, etc. For any observation $i$, the probability it does not get selected in one draw is $(n-1)/n$, so the probability it will not get selected in an entire sample of $n$ draws is $((n-1)/n)^n.$ Using that conclude, for a bootstrap sample of $n$ out of $n$ with return, roughly which proportion of the training samples get selected $0$ times?

 

1. This proportion increases with $n$ and reaches $1/2$ asymptotically

2. This proportion decreases with $n$ and reaches $0$ asymptotically

3. This proportion is about $1/e = 0.37$

4. This proportion is about $1/2$


#### Bonus question 
(send answers for this question seperately to saharon@tauex.tau.ac.il): 

roughly what proportion gets selected exactly one time, and how does that proportion depend on $n$? You can simulate for intuition, but to get a 5-point bonus for this HW grade, you should suggest a mathematical explanation. 

# finish!

to submit your HW please run this last code block and follow the instructions. <BR>
this code will create a CSV file in the current directory on the azure notebooks project <br>
please download it and submit it through moodle

In [ ]:
import pandas as pd
df_ans = pd.DataFrame.from_dict(ans, orient='index')
if df_ans.shape[0] == 14:
    df_ans.to_csv('{}_{}.csv'.format(ans['HW'],str(ans['id_number'])))
    print("OK!")
else:
    print("seems like you missed a question, make sure you have run all the code blocks")